In [49]:
import tensorflow

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

import pickle
import psycopg2

In [2]:
# Read data from PostGres SQL Data Base
with open("../database/secrets", "r") as file:
    secrets = [i.strip('\n') for i in file.readlines()]


def conn_curs():
    """
    makes a connection to the database dont worry these are dummy keys
    """

    connection = psycopg2.connect(dbname=secrets[4], user=secrets[4],
                                  password=secrets[5], host=secrets[6])
    cursor = connection.cursor()
    return connection, cursor


In [3]:
conn, curs = conn_curs()

In [4]:
df = pd.read_sql("SELECT * FROM posts", conn)

In [5]:
df.head()

,id,text,subreddit
0,1,Thousand Year Blood War Arc Anime Adaptation M...,bleach
1,2,Burn The Witch - Chapter 4 Discussion Thread #...,bleach
2,3,Let the journey begin.,bleach
3,4,"Since Ichigos an English Literature major, thi...",bleach
4,5,I just made these Ulquiorra customs for a clie...,bleach


In [7]:
le = LabelEncoder()
le.fit(df.subreddit.unique())

LabelEncoder()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.text, le.transform(df.subreddit), test_size=0.33, random_state=42)

In [9]:
vect = TfidfVectorizer(max_df=.95, min_df=80)

In [10]:
vect.fit(X_train)

TfidfVectorizer(max_df=0.95, min_df=80)

In [11]:
train_df = pd.DataFrame(vect.transform(X_train).todense(), columns=vect.get_feature_names())
train_df.head(), train_df.shape

(   000   10  100   11   12   13   14   15   16   17  ...  youll  young  \
 0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
 1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
 2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
 3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
 4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0   
 
    younger  your  youre  yourself  youtu  youtube  youve  zero  
 0      0.0   0.0    0.0       0.0    0.0      0.0    0.0   0.0  
 1      0.0   0.0    0.0       0.0    0.0      0.0    0.0   0.0  
 2      0.0   0.0    0.0       0.0    0.0      0.0    0.0   0.0  
 3      0.0   0.0    0.0       0.0    0.0      0.0    0.0   0.0  
 4      0.0   0.0    0.0       0.0    0.0      0.0    0.0   0.0  
 
 [5 rows x 1858 columns],
 (19957, 1858))

In [12]:
val_df = pd.DataFrame(vect.transform(X_test).todense(), columns=vect.get_feature_names())
val_df.head()

,000,10,100,11,12,13,14,15,16,17,...,youll,young,younger,your,youre,yourself,youtu,youtube,youve,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
embedding_dim=10
vocab_size = 10000

In [20]:
from tensorflow.keras.models import Sequential, Model # <- May Use
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Embedding
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Conv1D(filters=10, kernel_size=5, padding='same', activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters= 3,kernel_size=2,activation='relu'))
#model.add(LSTM(50, dropout=0.2))
model.add(Dense(50, activation='softmax') )
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(train_df.to_numpy(), y_train, batch_size=50, epochs=1, validation_data=(val_df.to_numpy(), y_test))

InvalidArgumentError:  assertion failed: [Condition x == y did not hold element-wise:] [x (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [50 1] [y (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [50 1858]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at <ipython-input-21-45d33ee7ebd4>:1) ]] [Op:__inference_train_function_1055]

Function call stack:
train_function
